In [3]:
from attrdict import AttrDict
from RL.ppo import *
from utils.utils import log
from envs.cpu.env import make_env
from envs.brax_custom.gpu_env import make_vec_env_brax
from models.actor_critic import QDActorCriticShared

from IPython.display import HTML, Image
from IPython.display import display
from brax.io import html, image
from brax import envs
from jax import numpy as jnp

pybullet build time: Dec 23 2020 01:49:05


In [4]:
%pwd
import os
os.chdir('/home/sbatra/QDPPO')

In [5]:
def enjoy_brax():
    cfg = {'env_name': 'ant', 'env_batch_size': None, 'normalize_obs': False, 'normalize_rewards': True,
           'num_dims': 4, 'envs_per_model': 1, 'seed': 0}
    cfg = AttrDict(cfg)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    env = make_vec_env_brax(cfg)

    obs_shape, action_shape = env.observation_space.shape, env.action_space.shape
    agent = QDActorCriticShared(cfg, obs_shape, action_shape, 4).to(device)
    cp_path = "checkpoints/model_0_checkpoint"
    model_state_dict = torch.load(cp_path)['model_state_dict']
    model_state_dict['_actor_logstd'] = model_state_dict['_actor_logstd'].reshape(1, -1)
    agent.load_state_dict(model_state_dict)

    obs = env.reset()
    rollout = [env.unwrapped._state]
    total_reward = 0
    done = False
    while not done:
        with torch.no_grad():
            obs = obs.unsqueeze(dim=0).to(device)
            act, _, _ = agent.get_action(obs)
            act = act.squeeze()
            obs, rew, done, info = env.step(act.cpu())
            print(rew)
            rollout.append(env.unwrapped._state)
            total_reward += rew

    i = HTML(html.render(env.unwrapped._env.sys, [s.qp for s in rollout]))
    display(i) 
    print(f'{total_reward=}')
    print(f' Rollout length: {len(rollout)}')

In [6]:
enjoy_brax()

tensor(-2.2409, device='cuda:0')
tensor(-1.6441, device='cuda:0')
tensor(1.0153, device='cuda:0')
tensor(-0.5053, device='cuda:0')
tensor(0.5439, device='cuda:0')
tensor(0.5813, device='cuda:0')
tensor(0.1866, device='cuda:0')
tensor(0.7032, device='cuda:0')
tensor(-1.4291, device='cuda:0')
tensor(0.6772, device='cuda:0')
tensor(2.2388, device='cuda:0')
tensor(0.4189, device='cuda:0')
tensor(-0.6553, device='cuda:0')
tensor(0.7555, device='cuda:0')
tensor(-1.2210, device='cuda:0')
tensor(-1.6683, device='cuda:0')
tensor(-1.0706, device='cuda:0')
tensor(0.2943, device='cuda:0')
tensor(1.4722, device='cuda:0')
tensor(-0.7796, device='cuda:0')
tensor(-1.7503, device='cuda:0')
tensor(-0.8479, device='cuda:0')
tensor(-1.7475, device='cuda:0')
tensor(-1.6876, device='cuda:0')


total_reward=tensor(-8.3603, device='cuda:0')
 Rollout length: 25
